In [1]:
import math
import numpy as np
import pandas
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# def load_data():
train_df = pandas.read_csv('../data/perros_train_v3.csv')
train_df

test_df = pandas.read_csv('../data/perros_predict_v3.csv')
# test_df.head()

# load_data()


In [3]:
y = train_df.Mascota
X = train_df.drop('Mascota', axis=1)

X.fillna("NaN", inplace=True)

In [4]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [5]:
print('Labels: {}'.format(set(y)))
print('Zero count = {}, One count = {}'.format(len(y) - sum(y), sum(y)))

Labels: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 4191, 4194, 4210, 4229, 4230, 4235, 4240, 4243, 4102, 4272, 4274, 4107, 4282, 4299, 4303, 4307, 4338, 4340, 4341, 4350, 4352, 4353, 4354, 4385, 4390, 4391, 4393, 4397, 4400, 4401, 4404, 4407, 4411, 4413, 4416, 4427, 4434, 4436, 4437, 4464, 4465, 4466, 4467, 4468, 4473, 4478, 4480, 4491, 4492, 4502, 4512, 4515, 4516, 4531, 4535, 4537, 4539, 4541, 4546, 4547, 4551, 4642, 1000, 1002, 1003, 1007, 1009, 1010, 1014, 1018, 1036, 1048, 1053, 1066, 1067, 1074, 1081, 1082, 1083, 1095, 1100, 1116, 1121, 1122, 1129, 1136, 1144, 1157, 1166, 1174, 1183, 1186, 1191, 1207, 1215, 1217, 1219, 1223, 1224, 1227, 1232, 1235, 1238, 1279, 1329, 1343, 1360, 1372, 1373, 1377, 1381, 1382, 1385, 1387, 1395, 1399, 1401, 1409, 1431, 1432, 

In [6]:
pool3 = Pool(data=X, cat_features=cat_features)

print('Dataset shape')
print('dataset 3:' + str(pool3.shape))

print('\n')
print('Column names')
print('\ndataset 3:')
print(pool3.get_feature_names())


Dataset shape
dataset 3:(320, 13)


Column names

dataset 3:
['Edad', 'Tamaño', 'Sexo', 'Patron de pelaje', 'Color de pelaje 1', 'Color de pelaje 2', 'Color de pelaje 3', 'Largo de pelaje', 'Color de ojos', 'Largo de hocico', 'Largo de cola', 'Largo de orejas', 'Tipo de orejas']


In [7]:
from sklearn.model_selection import train_test_split
# X_train, X_validation, y_train, y_validation = train_test_split(
#     X, y, train_size=0.8, random_state=1234)


validation_df = pandas.read_csv('../data/perros_validation_v3.csv')
y_validation = train_df.Mascota
X_validation = train_df.drop('Mascota', axis=1)
X_validation.fillna("NaN", inplace=True)

X_train = X
y_train = y 


In [17]:
model = CatBoostClassifier(
    class_names=y,
    random_seed=63,
    iterations=3000,
    cat_features=cat_features,
    # learning_rate=0.25,
    one_hot_max_size=4,
    depth=5,
    loss_function='MultiClass',
    use_best_model=True
)


In [ ]:
''' model = CatBoostClassifier(
    iterations=100,
    depth=3,
    loss_function='MultiClass'
) '''

model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    # verbose=False
)

print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())


In [12]:
model.save_model("../models/withValidation/modelD5it2000.cbm",
                    format="cbm",
                    export_parameters=None,
                    pool=None)

In [13]:
print(model.get_feature_importance(prettified=True))

           Feature Id  Importances
0   Color de pelaje 1    28.721838
1   Color de pelaje 2    23.091965
2     Largo de orejas    19.575626
3   Color de pelaje 3    18.669368
4              Tamaño     2.949186
5       Largo de cola     1.804431
6     Largo de hocico     1.273351
7     Largo de pelaje     1.271970
8      Tipo de orejas     0.830115
9                Sexo     0.643835
10      Color de ojos     0.615630
11               Edad     0.429147
12   Patron de pelaje     0.123539


In [58]:
model2 = CatBoostClassifier(
    random_seed=63,
    iterations=1000,
    # learning_rate=0.25,
    depth=5,
    loss_function='MultiClass',
    use_best_model=True
)

model2.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    init_model=model
    # verbose=False
)

print('Model 2 is fitted: ' + str(model2.is_fitted()))
print('Model 2 params:')
print(model2.get_params())


Learning rate set to 0.217853
0:	learn: 5.2046555	test: 5.2134757	best: 5.2134757 (0)	total: 7.7s	remaining: 19m 6s
1:	learn: 5.1634807	test: 5.2120469	best: 5.2120469 (1)	total: 14.7s	remaining: 18m 8s
2:	learn: 5.1453802	test: 5.2108142	best: 5.2108142 (2)	total: 22.7s	remaining: 18m 33s
3:	learn: 5.1177736	test: 5.2034173	best: 5.2034173 (3)	total: 30.6s	remaining: 18m 35s
4:	learn: 5.1065871	test: 5.1993662	best: 5.1993662 (4)	total: 37.3s	remaining: 18m 2s
5:	learn: 5.0825682	test: 5.1950779	best: 5.1950779 (5)	total: 43.7s	remaining: 17m 29s
6:	learn: 5.0498946	test: 5.1881394	best: 5.1881394 (6)	total: 50.4s	remaining: 17m 8s
7:	learn: 5.0382427	test: 5.1839985	best: 5.1839985 (7)	total: 57.6s	remaining: 17m 2s
8:	learn: 5.0219381	test: 5.1797768	best: 5.1797768 (8)	total: 1m 4s	remaining: 16m 46s
9:	learn: 5.0060330	test: 5.1758371	best: 5.1758371 (9)	total: 1m 10s	remaining: 16m 33s
10:	learn: 4.9822763	test: 5.1725216	best: 5.1725216 (10)	total: 1m 17s	remaining: 16m 21s
11:	

In [25]:
validation_df = pandas.read_csv('../data/perros_predict_v3.csv')
y_validation = train_df.Mascota
X_validation = train_df.drop('Mascota', axis=1)
X_validation.fillna("NaN", inplace=True)

X_train = X
y_train = y
# X_validation =
print(model.predict_proba(X_validation))


[[0.00315492 0.00318327 0.00318327 ... 0.00318327 0.0031294  0.00310324]
 [0.00315492 0.00318327 0.00318327 ... 0.00318327 0.0031294  0.00310324]
 [0.00315492 0.00318327 0.00318327 ... 0.00318327 0.0031294  0.00310324]
 ...
 [0.00315492 0.00318327 0.00318327 ... 0.00318327 0.0031294  0.00310324]
 [0.00314886 0.00312168 0.00312168 ... 0.00312168 0.00312339 0.00312365]
 [0.00315492 0.00318327 0.00318327 ... 0.00318327 0.0031294  0.00310324]]


In [60]:
print(model.predict(X_validation))

[[1157]
 [1157]
 [1157]
 [1157]
 [  34]
 [1074]
 [1157]
 [  34]
 [1074]
 [4512]
 [1157]
 [  12]
 [  13]
 [1470]
 [  34]
 [  16]
 [1074]
 [1279]
 [  12]
 [  34]
 [  34]
 [  12]
 [  34]
 [  34]
 [4512]
 [  34]
 [6064]
 [1279]
 [1074]
 [  34]
 [  31]
 [1800]
 [4307]
 [  34]
 [1157]
 [6065]
 [1470]
 [1526]
 [  39]
 [  34]
 [1157]
 [6065]
 [  13]
 [1526]
 [1074]
 [1074]
 [6065]
 [1279]
 [  34]
 [  34]
 [1074]
 [  34]
 [1157]
 [  34]
 [1470]
 [4512]
 [1183]
 [1470]
 [6065]
 [1800]
 [1074]
 [6065]
 [6065]
 [6065]
 [1800]
 [1279]
 [1470]
 [1144]
 [1036]
 [1074]
 [1800]
 [1470]
 [6065]
 [1800]
 [1401]
 [4512]
 [1066]
 [  34]
 [6065]
 [6065]
 [  34]
 [1074]
 [  34]
 [  34]
 [1157]
 [1800]
 [1279]
 [1401]
 [1800]
 [1800]
 [1401]
 [1144]
 [1800]
 [3507]
 [4307]
 [  34]
 [3631]
 [  34]
 [  34]
 [1761]
 [  34]
 [1800]
 [3592]
 [1144]
 [1640]
 [1157]
 [1800]
 [3631]
 [1144]
 [4512]
 [1800]
 [1800]
 [  34]
 [1183]
 [3524]
 [1401]
 [  34]
 [1470]
 [1401]
 [1470]
 [1279]
 [1215]
 [6065]
 [1800]
 [6064]


In [61]:
prediction1 = model.get_feature_importance(prettified=True)
prediction1

,Feature Id,Importances
0,Largo de pelaje,37.479364
1,Color de ojos,16.002910
2,Color de pelaje 3,14.736305
3,Tamaño,14.159547
4,Color de pelaje 1,11.721368
5,Largo de hocico,4.931495
6,Edad,0.969011
7,Sexo,0.000000
8,Patron de pelaje,0.000000
9,Color de pelaje 2,0.000000


In [62]:
prediction2 = model2.get_feature_importance(prettified=True)
prediction2

,Feature Id,Importances
0,Largo de pelaje,31.113495
1,Color de ojos,19.111838
2,Color de pelaje 3,18.343051
3,Tamaño,15.203147
4,Color de pelaje 1,13.274599
5,Largo de hocico,2.468770
6,Edad,0.485100
7,Sexo,0.000000
8,Patron de pelaje,0.000000
9,Color de pelaje 2,0.000000


In [59]:
prediction3 = model2.get_feature_importance(prettified=True)
prediction3

,Feature Id,Importances
0,Largo de pelaje,31.113495
1,Color de ojos,19.111838
2,Color de pelaje 3,18.343051
3,Tamaño,15.203147
4,Color de pelaje 1,13.274599
5,Largo de hocico,2.468770
6,Edad,0.485100
7,Sexo,0.000000
8,Patron de pelaje,0.000000
9,Color de pelaje 2,0.000000


In [65]:
# y = train_df.Mascota
test_df = train_df.drop('Mascota', axis=1)

test_df.fillna("NaN", inplace=True)

In [68]:
# print(model.predict_proba(test_df))
print(model2.predict(test_df))

[[1401]
 [1157]
 [  13]
 [1157]
 [   5]
 [1074]
 [1401]
 [  34]
 [1074]
 [3914]
 [1157]
 [6040]
 [  13]
 [  13]
 [  34]
 [  16]
 [1074]
 [1279]
 [6040]
 [3507]
 [3507]
 [6040]
 [  34]
 [  34]
 [4512]
 [  34]
 [1526]
 [1279]
 [6066]
 [  34]
 [  38]
 [1800]
 [  16]
 [  34]
 [1157]
 [6065]
 [  13]
 [1526]
 [  39]
 [  34]
 [1157]
 [4512]
 [  13]
 [1526]
 [1074]
 [1074]
 [4512]
 [1279]
 [3507]
 [  34]
 [1074]
 [4243]
 [1157]
 [4537]
 [1401]
 [4512]
 [1183]
 [  13]
 [6065]
 [1800]
 [1074]
 [4512]
 [4512]
 [6065]
 [1800]
 [1279]
 [1401]
 [3507]
 [1036]
 [1074]
 [6066]
 [1401]
 [6065]
 [1800]
 [1401]
 [4512]
 [1066]
 [4537]
 [4512]
 [4512]
 [  34]
 [1074]
 [  34]
 [  34]
 [1157]
 [6072]
 [1279]
 [1401]
 [3507]
 [3507]
 [1401]
 [6066]
 [6066]
 [3507]
 [3524]
 [  34]
 [3524]
 [  34]
 [  34]
 [6040]
 [  34]
 [3507]
 [3592]
 [6066]
 [3595]
 [1157]
 [6066]
 [3524]
 [3507]
 [4512]
 [1800]
 [4243]
 [4537]
 [1183]
 [1486]
 [1401]
 [  34]
 [1470]
 [1401]
 [1470]
 [1279]
 [1215]
 [3914]
 [1800]
 [1217]
